In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import multiprocessing

from scipy.stats import randint

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.tree import DecisionTreeRegressor, plot_tree

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

import tftinsights
from tftinsights.data import get_tft_dataset

# Placement Regression 

In this notebook, we'll aim to build a regression model for a certain composition
The goal is to extract information about what influnces win rates, and to determien if we need to feature engineer anything.



In [3]:

#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
df = tftinsights.data.get_tft_dataset(include_traits  =True, include_units =True, include_items=False,include_player_data=False)  
print(list(df.keys())) #remaining columns

['trait_TFT14_AnimaSquad', 'trait_TFT14_Armorclad', 'trait_TFT14_BallisTek', 'trait_TFT14_Bruiser', 'trait_TFT14_Controller', 'trait_TFT14_Cutter', 'trait_TFT14_Cyberboss', 'trait_TFT14_Divinicorp', 'trait_TFT14_EdgeRunner', 'trait_TFT14_HotRod', 'trait_TFT14_Immortal', 'trait_TFT14_Marksman', 'trait_TFT14_Mob', 'trait_TFT14_Netgod', 'trait_TFT14_Overlord', 'trait_TFT14_StreetDemon', 'trait_TFT14_Strong', 'trait_TFT14_Suits', 'trait_TFT14_Supercharge', 'trait_TFT14_Swift', 'trait_TFT14_Techie', 'trait_TFT14_Thirsty', 'trait_TFT14_Vanguard', 'trait_TFT14_ViegoUniqueTrait', 'trait_TFT14_Virus', 'unit_TFT14_Alistar', 'unit_TFT14_Annie', 'unit_TFT14_Aphelios', 'unit_TFT14_Aurora', 'unit_TFT14_Brand', 'unit_TFT14_Braum', 'unit_TFT14_Chogath', 'unit_TFT14_Darius', 'unit_TFT14_DrMundo', 'unit_TFT14_Draven', 'unit_TFT14_Ekko', 'unit_TFT14_Elise', 'unit_TFT14_Fiddlesticks', 'unit_TFT14_Galio', 'unit_TFT14_Garen', 'unit_TFT14_Gragas', 'unit_TFT14_Graves', 'unit_TFT14_Illaoi', 'unit_TFT14_Jarvan'

In [4]:
# #Quick Validation
print ( df[df['unit_TFT14_Kobuko']==3.0]['placement'].mean() )
#Getting a single 3* kobuko will very likely win you the game, which is expected. 
#We shohuld expect a number between 1 and 2, very close to 1. 

1.1111111111111112


In [5]:
def TFTDecisionTreeRegression ( df ):
    import numpy as np
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.model_selection import RandomizedSearchCV, KFold
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from scipy.stats import randint, uniform
    import warnings
    import multiprocessing # my CPU > GPU

    y = df['placement'].astype(float) #dependent variable
    x = df.drop(columns='placement')    #independent variables 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3126)

    param_distributions = {
        'max_depth': randint(5, 700),  # integer range
        'min_samples_split': uniform(0.1,0.7),  # float between 0.1 and 0.4
        'min_samples_leaf': randint(1, 500)   # float between 0.1 and 0.3
    }
    
    # Set up the model
    model = DecisionTreeRegressor(random_state=3126)
    
    # Random search setup
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=1000,
        scoring='neg_root_mean_squared_error',
        cv=5,
        random_state=42,
        verbose=1,
        n_jobs=-multiprocessing.cpu_count() - 3
    )
    
    # Run random search
    search.fit(x_train, y_train)
    
    # Best estimator
    best_model = search.best_estimator_
    best_params = search.best_params_
    print("\nBest Params:", best_params)
    
    y_pred = best_model.predict(x_test)
    
    # Evaluation
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nModel Evaluation:")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"R²:  {r2:.4f}")

    return search

## Placement Regression: Traits + Units

In [6]:
print(df.shape)
search = TFTDecisionTreeRegression ( df )

(140216, 86)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

Model Evaluation:
MAE: 1.5497
MSE: 3.5166
R²:  0.3281


In [7]:
from sklearn.tree import export_text
x = df.drop(columns='placement')    #independent variables 
best_tree = search.best_estimator_
tree_rules = export_text(best_tree, feature_names=x.columns.tolist())
print(tree_rules)

|--- unit_TFT14_Kobuko <= 1.50
|   |--- unit_TFT14_Aurora <= 1.50
|   |   |--- unit_TFT14_Samira <= 1.50
|   |   |   |--- unit_TFT14_Renekton <= 1.50
|   |   |   |   |--- unit_TFT14_Braum <= 2.50
|   |   |   |   |   |--- unit_TFT14_Viego <= 1.50
|   |   |   |   |   |   |--- unit_TFT14_Neeko <= 1.50
|   |   |   |   |   |   |   |--- unit_TFT14_Leona <= 1.50
|   |   |   |   |   |   |   |   |--- unit_TFT14_Sejuani <= 1.50
|   |   |   |   |   |   |   |   |   |--- unit_TFT14_Chogath <= 1.50
|   |   |   |   |   |   |   |   |   |   |--- unit_TFT14_Zed <= 1.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 29
|   |   |   |   |   |   |   |   |   |   |--- unit_TFT14_Zed >  1.50
|   |   |   |   |   |   |   |   |   |   |   |--- value: [4.42]
|   |   |   |   |   |   |   |   |   |--- unit_TFT14_Chogath >  1.50
|   |   |   |   |   |   |   |   |   |   |--- value: [4.43]
|   |   |   |   |   |   |   |   |--- unit_TFT14_Sejuani >  1.50
|   |   |   |   |   |   |   |   |   |--- v

# As you can see, the regression here is pretty weak since at about 0.35ish for R2. 
# Taking a look at the decision tree breakdown, we can see it is just splitting off and looking at unit the player have (at a float, why not int?) 
# So this probably indicates we should do some amount of feature engineering to just group up the number of units by cost. 

Before we do feature engineering, lets see what set of information is sufficent for the regression;

Traits,
Units
Items

Or what combination of it works best,


## Placement Regression: Traits ONLY

In [8]:
df = tftinsights.data.get_tft_dataset(include_traits=True, include_units=False, include_items=False, include_player_data=False)  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (140216, 26)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

# Model Evaluation:
# MAE: 1.7771
# MSE: 4.4129

(140216, 26)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

Model Evaluation:
MAE: 1.7771
MSE: 4.4129
R²:  0.1568


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D674346D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D67435A20>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D67437F40>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

## Placement Regression: Units ONLY

In [9]:
df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False)  
print(df.shape)
TFTDecisionTreeRegression ( df )

(140216, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

Model Evaluation:
MAE: 1.5571
MSE: 3.5369
R²:  0.3242


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D653E1060>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D653E1A50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D653E3A60>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

WE got results similar to our initial regression with TRAITS + UNITS;

<b>This indicates that the decision tree is able to tell the relationships between unit syngergies.</b>

Meaning traits are not great for generalization.


## Placement Regression: Items ONLY

In [10]:
df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=False, include_items=True, include_player_data=False)  
print(df.shape)
TFTDecisionTreeRegression ( df )

(140216, 165)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 555, 'min_samples_leaf': 415, 'min_samples_split': 0.10078407803607664}

Model Evaluation:
MAE: 1.8370
MSE: 4.6879
R²:  0.1043


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D674342B0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D674374F0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D67435F00>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

## Placement Regression: Units + Items

In [23]:
df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=True, include_player_data=False)  
print(df.shape)
TFTDecisionTreeRegression ( df )

(140216, 225)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

Model Evaluation:
MAE: 1.5639
MSE: 3.5593
R²:  0.3199


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65DDAC20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65DDBD90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D65DD9390>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

Adding items units to do the regression produced no noticable results, indicatingt that units is the strongest predictor of success.
This assumes the player is good enough to not combine bad items on the wrong unit. 


Items placement is known to be critical in the game
so how can we get that data, or how can  we help the model to understand that importnat? 

### Model Evaluation Metrics (Sorted by R²)

| Model           | MAE    | MSE    | R²     |
|----------------|--------|--------|--------|
| Trait + Unit   | 1.5534 | 3.5230 | 0.3269 |
| Units Only     | 1.5571 | 3.5369 | 0.3242 |
| Unit + Items   | 1.5639 | 3.5593 | 0.3199 |
| Trait Only     | 1.7821 | 4.4365 | 0.1523 |
| Items Only     | 1.8370 | 4.6879 | 0.1043 |

Given the behaviors observed in our data set, Unit is the best subset predictor of Placements.
Items isn't relevant because those that win with a composition will have nautrally required to have the correct items for it, espeically the higher the player goes up in ranking. This is something that the ML is unlikely able to interpret.

Looking at the decision tree from Traits+Unit, we can see the decision branch notes are all 5 and 4 cost units, which seems to suggest we should feature engineer # of X $ units columns - # of 1 cost unit, # of 2 cost Unit, etc etc. 

# Skill Subset


In [24]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="IRON")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (346, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 361, 'min_samples_leaf': 23, 'min_samples_split': 0.13364124693792362}

# Model Evaluation:
# MAE: 1.5158
# MSE: 2.8924
# R²:  0.0388

(346, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 361, 'min_samples_leaf': 23, 'min_samples_split': 0.13364124693792362}

Model Evaluation:
MAE: 1.5158
MSE: 2.8924
R²:  0.0388


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D653DE4A0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D74A4B2E0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D6547A020>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [25]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="BRONZE")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (3365, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 663, 'min_samples_leaf': 4, 'min_samples_split': 0.1170211765020177}

# Model Evaluation:
# MAE: 1.4435
# MSE: 3.2218
# R²:  0.2309

(3365, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 663, 'min_samples_leaf': 4, 'min_samples_split': 0.1170211765020177}

Model Evaluation:
MAE: 1.4435
MSE: 3.2218
R²:  0.2309


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5E963010>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65DD9300>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D5E961E10>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [26]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="SILVER")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (17137, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 663, 'min_samples_leaf': 4, 'min_samples_split': 0.1170211765020177}

# Model Evaluation:
# MAE: 1.5374
# MSE: 3.4522
# R²:  0.3315

(17137, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 663, 'min_samples_leaf': 4, 'min_samples_split': 0.1170211765020177}

Model Evaluation:
MAE: 1.5374
MSE: 3.4522
R²:  0.3315


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D601D2410>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5F093190>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D5EA23820>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [27]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="GOLD")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (36992, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

# Model Evaluation:
# MAE: 1.5217
# MSE: 3.4523
# R²:  0.3353

(36992, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

Model Evaluation:
MAE: 1.5217
MSE: 3.4523
R²:  0.3353


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65DA8D90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65DABA90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D65DD9ED0>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [28]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="PLATINUM")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (39138, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

# Model Evaluation:
# MAE: 1.5178
# MSE: 3.4207
# R²:  0.3490

(39138, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

Model Evaluation:
MAE: 1.5178
MSE: 3.4207
R²:  0.3490


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5FE1FF10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5F0934C0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D65475990>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [29]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="EMERALD")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (21570, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

# Model Evaluation:
# MAE: 1.5517
# MSE: 3.5342
# R²:  0.3243

(21570, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 463, 'min_samples_leaf': 58, 'min_samples_split': 0.10072063589194338}

Model Evaluation:
MAE: 1.5517
MSE: 3.5342
R²:  0.3243


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5E963190>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5E960BB0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D653DDF30>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [30]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="DIAMOND")  
print(df.shape)
TFTDecisionTreeRegression ( df )

# (8620, 61)
# Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

# Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

# Model Evaluation:
# MAE: 1.5611
# MSE: 3.6520
# R²:  0.3120

(8620, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

Model Evaluation:
MAE: 1.5611
MSE: 3.6520
R²:  0.3120


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65D17B80>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D65D17A00>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D67435EA0>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [31]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="MASTER")  
print(df.shape)
TFTDecisionTreeRegression ( df )

(3956, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 35, 'min_samples_leaf': 23, 'min_samples_split': 0.10360161952830034}

Model Evaluation:
MAE: 1.5721
MSE: 3.7097
R²:  0.2814


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5E88FCA0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D601D1570>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D5E962920>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [ ]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER' # ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="GRANDMASTER")  
print(df.shape)
TFTDecisionTreeRegression ( df )

(762, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 412, 'min_samples_leaf': 13, 'min_samples_split': 0.15711592628016824}

Model Evaluation:
MAE: 1.6981
MSE: 4.5149
R²:  0.0779


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5F092C20>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D74A49B70>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D6534ED40>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [33]:
#def get_tft_dataset ( include_traits  =True, include_units =True, include_items =True, include_units_item=False, ranked_queue=True, include_player_data=True, rankFilter=""):
# tier_order = [
#     'IRON', 'BRONZE', 'SILVER', 'GOLD',
#     'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 
#     'GRANDMASTER', 'CHALLENGER'
# ]

df = tftinsights.data.get_tft_dataset(include_traits=False, include_units=True, include_items=False, include_player_data=False, rankFilter="CHALLENGER")  
print(df.shape)
TFTDecisionTreeRegression ( df )

(417, 61)
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits

Best Params: {'max_depth': 353, 'min_samples_leaf': 8, 'min_samples_split': 0.11896719475243844}

Model Evaluation:
MAE: 1.7013
MSE: 4.5856
R²:  0.1009


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=3126),
                   n_iter=1000, n_jobs=-19,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D5E88E230>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025D6A5334F0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025D5E963490>},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)